# Rcnn-Instance-Segmentation

In [ ]:
# Basics
from glob import glob # finds pathnames
import os # Miscellaneous operating system interfaces
import sys
import random
import timeit
import imp
import gc

import numpy as np 
import pandas as pd 
from scipy.ndimage import label as scipy_label
from scipy.ndimage import generate_binary_structure
from dices import *

from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log

import matplotlib.pyplot as plt

from steel_dataset import SteelDataset
from steel_config import s

In [ ]:
ROOT_DIR = './input/'
Train_Dir = ROOT_DIR + 'train_images/'
Test_Dir = ROOT_DIR + 'test_images/'

MODEL_DIR = 'logs'

# trained weights file
# Trained_Weights = 'logs/steel20191114T0608/mask_rcnn_steel_0066.h5'

In [ ]:
config = SteelConfig()
config.display()

In [ ]:
# select files for test and validation dataset
annotations_train = pd.read_csv(ROOT_DIR + 'train_1.csv')
annotations_test = pd.read_csv(ROOT_DIR + 'val_1.csv')

train = annotations_train['ImageId'].unique().copy()
val = annotations_test['ImageId'].unique().copy()

In [ ]:
dataset_train = SteelDataset()
dataset_train.load_steel(ROOT_DIR, train)
dataset_train.prepare()

dataset_val = SteelDataset()
dataset_val.load_steel(ROOT_DIR, val)
dataset_val.prepare()

model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
model.keras_model.metrics_tensors = []

In [ ]:
# Train model
timestart = timeit.default_timer()

model.train(dataset_train, 
            dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=config.EPOCHS,
            layers='all')

In [ ]:
# create model for inferring
infer_config = SteelConfig()
infer_config.BATCH_SIZE=1
infer_config.IMAGES_PER_GPU=1
infer_config.DETECTION_MIN_CONFIDENCE=.5
infer_config.DETECTION_NMS_THRESHOLD=.5
model_infer = modellib.MaskRCNN(mode="inference", config=infer_config, model_dir=MODEL_DIR)
model_infer.load_weights('logs/steel20191209T0236/mask_rcnn_steel_0091.h5', by_name=True)

In [ ]:
from tqdm import tqdm
# calculate validation set positive and negative indices
pos = 0
neg = 0
for image_id in tqdm(range(500)):
# for image_id in range(3):
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    rr = model_infer.detect([image])
    r = rr[0]
    score = get_score(mask, class_ids, r)
    pos += score[0]
    neg += score[1]
pos /= len(val)
neg /= len(val)